In [7]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# cp_callback = tf.keras.callbacks.ModelCheckpoint("./checkpoints/checkpoint-1", 
#                                                  save_weights_only=False,
#                                                  verbose=1)

# model.fit(x_train, y_train, epochs=5, callbacks=[cp_callback])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

print(model.outputs)

Epoch 1/5
59872/60000 [============================>.] - ETA: 0s - loss: 0.2183 - acc: 0.9355
Epoch 00001: saving model to ./checkpoints/checkpoint-1
60000/60000 [==============================] - 11s 181us/sample - loss: 0.2180 - acc: 0.9355
Epoch 2/5
59840/60000 [============================>.] - ETA: 0s - loss: 0.0970 - acc: 0.9702
Epoch 00002: saving model to ./checkpoints/checkpoint-1
60000/60000 [==============================] - 11s 189us/sample - loss: 0.0972 - acc: 0.9701
Epoch 3/5
59936/60000 [============================>.] - ETA: 0s - loss: 0.0699 - acc: 0.9775
Epoch 00003: saving model to ./checkpoints/checkpoint-1
60000/60000 [==============================] - 12s 195us/sample - loss: 0.0700 - acc: 0.9775
Epoch 4/5
59744/60000 [============================>.] - ETA: 0s - loss: 0.0544 - acc: 0.9825
Epoch 00004: saving model to ./checkpoints/checkpoint-1
60000/60000 [==============================] - 11s 190us/sample - loss: 0.0543 - acc: 0.9825
Epoch 5/5
59808/60000 [=====

In [8]:
# save model to pb ====================
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [9]:
from keras import backend as K
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, ".", "ck.pb", as_text=True)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph


Using TensorFlow backend.


FailedPreconditionError: Error while reading resource variable training_8/Adam/Variable_9 from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/training_8/Adam/Variable_9)
	 [[node training_8/Adam/Variable_9/Read/ReadVariableOp (defined at <ipython-input-5-daefae591910>:21) ]]
	 [[node dense_3/bias/Read/ReadVariableOp (defined at <ipython-input-2-9d541136d9ce>:11) ]]

Caused by op 'training_8/Adam/Variable_9/Read/ReadVariableOp', defined at:
  File "/home/ernest/anaconda3/envs/RoboticsEnv/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ernest/anaconda3/envs/RoboticsEnv/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ernest/anaconda3/envs/RoboticsEnv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ernest/anaconda3/envs/Rob